### Disasters on social media

https://github.com/hundredblocks/concrete_NLP_tutorial/blob/master/NLP_notebook.ipynb

Data: https://d1p17r2m4rzlbo.cloudfront.net/wp-content/uploads/2016/03/socialmedia-disaster-tweets-DFE.csv

GOAL: parse tweets to find out which are about disasters, depending on the keywords

In [14]:
import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs

Remove symbols from text & make it readable. Also convert texts to lowercase 

In [15]:
def purify_chars(raw_data_file, save_to):
    output_file = open(save_to, "w")
    input_file = codecs.open(raw_data_file, 'r', errors='replace', encoding='utf-8')
    for line in input_file:
        output_file.write(line)
    output_file.close()

In [16]:
purify_chars("socialmedia-disaster-tweets-DFE.csv", "socialmedia_cols_clean.csv")

In [18]:
df = pd.read_csv("socialmedia_cols_clean.csv", names=['text', 'label_c', 'label'])
df.sample(10)

,,,,,,,,,,text,label_c,label
778247818,FALSE,finalized,5,8/28/15 4:29,Not Relevant,0.7988,NaN,dead,"Colwyn Bay, Wales",WHAT AN INCREDIBLE CHARACTER MY HEART IS BROKE...,6.2936E+17,31747428
778251776,FALSE,finalized,5,8/29/15 0:24,Relevant,1,NaN,massacre,Colorado,Colorado movie massacre trial jurors reach ver...,6.29077E+17,116465507
778254025,FALSE,finalized,5,8/28/15 20:38,Relevant,1,NaN,suicide%20bombing,NaN,'Suicide bombing at [location named]...' #prem...,6.28858E+17,2692189418
778246338,FALSE,finalized,5,9/2/15 8:57,Not Relevant,1,NaN,body%20bags,NaN,Womens Handbags Cross Body Geometric Pattern S...,6.2902E+17,3071663170
778246018,FALSE,finalized,5,9/1/15 13:36,Not Relevant,0.8,NaN,blizzard,NaN,Tomorrow's Announcement VODs http://t.co/cUbze...,6.29106E+17,3314173138
778250529,FALSE,finalized,5,8/29/15 12:05,Relevant,1,NaN,forest%20fire,PDX,BE CAREFUL anyone who lives west of Beaverton....,6.29082E+17,1360507861
778255208,FALSE,finalized,5,8/28/15 3:35,Relevant,0.6006,NaN,whirlwind,CHICAGO,A quarter whirlwind. They don't see it coming.,6.29321E+17,196327794
778244921,FALSE,finalized,6,8/26/15 16:25,Relevant,0.6672,NaN,accident,Australia,BigRigRadio Live Accident Awareness,6.29353E+17,47760908
778245089,FALSE,finalized,5,9/1/15 19:13,Not Relevant,1,NaN,ambulance,Loveland Colorado,@Kiwi_Karyn Check out what's in my parking lot...,6.29343E+17,2919060905
778250563,FALSE,finalized,5,8/28/15 7:47,Relevant,0.8017,NaN,forest%20fires,Montana,The forest service now spends &gt;50% of its b...,6.29057E+17,69365437
